In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

***SEMANA 1***

In [ ]:
#importamos el dia con el que trabajamos
df_sem = pd.read_csv('../data/serie_temporal_semana1.csv')

In [ ]:
# Convertimos el timestamp en índice de tiempo
df_sem['user_ts__'] = pd.to_datetime(df_sem['user_ts__'])
df_sem.set_index('user_ts__', inplace=True)

In [ ]:
# Revisamos cantidad de datos no nulos por columna
non_null_counts_sem = df_sem.notnull().sum().sort_values(ascending=False)
non_null_counts_sem

In [ ]:
# Gráfico de cantidad de datos por columna
plt.figure(figsize=(12, 12))
non_null_counts_sem.plot(kind='bar', color='orange', edgecolor='black')
plt.title('Cantidad de datos no nulos por columna de temperatura')
plt.xlabel('Variables')
plt.ylabel('Cantidad de datos no nulos')
plt.xticks(rotation=90)
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Seleccionamos la variable con más datos
serie = df_sem['powerPerPreform_CurrentPreformNeckFinishTemperature.0'].dropna()

# Graficamos la serie temporal
plt.figure(figsize=(14, 6))
plt.plot(serie, color='blue')

plt.title('Serie temporal - powerPerPreform_CurrentPreformNeckFinishTemperature.0', fontsize=16)
plt.xlabel('Fecha y hora', fontsize=14)
plt.ylabel('Temperatura', fontsize=14)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# 1. Aplicar el test de Dickey-Fuller
resultado_dickey = adfuller(serie.dropna(), autolag='AIC')

# 2. Crear una serie de resultados básicos
resultado_df = pd.Series(resultado_dickey[0:4], index=[
    'Estadístico de prueba',
    'p-value',
    'Número de rezagos usados',
    'Número de observaciones usadas'
])

# 3. Agregar los valores críticos al resultado
for clave, valor in resultado_dickey[4].items():
    resultado_df[f'Valor crítico ({clave})'] = valor

# 4. Mostrar el resultado completo
print(resultado_df)

In [ ]:
# Creamos una figura con 2 subplots para ACF y PACF
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# ACF: para definir q
plot_acf(serie.dropna(), lags=50, ax=axes[0])
axes[0].set_title('ACF - Función de Autocorrelación')

# PACF: para definir p
plot_pacf(serie.dropna(), lags=50, ax=axes[1], method='ywm')
axes[1].set_title('PACF - Función de Autocorrelación Parcial')

plt.show()

In [ ]:
from pmdarima import auto_arima

# Asumiendo que serie es la serie semanal ya cargada y preprocesada
modelo_auto = auto_arima(serie,
                         start_p=0, start_q=0,
                         max_p=5, max_q=5,
                         seasonal=False,     # SIN estacionalidad aquí
                         stepwise=True,
                         trace=True,
                         suppress_warnings=True)

# Mostramos el resultado del modelo sin estacionalidad
print(modelo_auto.summary())
print('Orden sin estacionalidad sugerido:', modelo_auto.order)

# Lo guardas para el siguiente paso
best_order = modelo_auto.order

In [ ]:
# Asegúrate de tener el índice en datetime y ordenado
serie.index = pd.to_datetime(serie.index)
serie = serie.sort_index()

# Resampleamos cada minuto y calculamos el promedio en cada ventana de 1 min
serie_minuto = serie.resample('1min').mean().dropna()

# Revisamos
print(f"Cantidad de datos resampleados: {len(serie_minuto)}")
serie_minuto.head()

In [ ]:
# Asegúrate de tener el índice en datetime y ordenado
serie.index = pd.to_datetime(serie.index)
serie = serie.sort_index()

# Resampleamos cada 20 segundos y calculamos el promedio en cada ventana de 20 segundos
serie_30s = serie.resample('30S').mean().dropna()

# Revisamos
print(f"Cantidad de datos resampleados: {len(serie_30s)}")
serie_30s.head()

In [ ]:
from statsmodels.tsa.stattools import adfuller

resultado_adf = adfuller(serie_minuto)
print(f'Estadístico de prueba: {resultado_adf[0]}')
print(f'p-value: {resultado_adf[1]}')

# Si p-value < 0.05 => estacionaria

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import itertools

fixed_order = (4, 1, 2)  # El orden sin estacionalidad que obtuviste

# Rango de búsqueda para estacionalidad
P = D = Q = range(0, 2)  # Ajustable
m = 60  # Un ciclo diario

seasonal_combinations = list(itertools.product(P, D, Q))

best_aic = float("inf")
best_model = None
best_seasonal_order = None

for seasonal_order in seasonal_combinations:
    try:
        model = SARIMAX(serie_minuto,  # O el nombre de tu serie de 1 min
                        order=fixed_order,
                        seasonal_order=seasonal_order + (m,),
                        enforce_stationarity=False,
                        enforce_invertibility=False)

        result = model.fit(disp=False)

        print(f'Tested SARIMA{fixed_order}x{seasonal_order + (m,)} - AIC:{result.aic}')

        if result.aic < best_aic:
            best_aic = result.aic
            best_model = result
            best_seasonal_order = seasonal_order + (m,)

    except Exception as e:
        print(f'Error en SARIMA{fixed_order}x{seasonal_order + (m,)}: {e}')

print(f'\n📌 Mejor modelo encontrado:')
print(f'  - SARIMA{fixed_order}x{best_seasonal_order} con AIC = {best_aic}')

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Entrena el modelo con los mejores parámetros
final_model = SARIMAX(serie_30s,
                      order=(4, 1, 2),
                      seasonal_order=(0, 1, 2, 60),
                      enforce_stationarity=False,
                      enforce_invertibility=False)

final_result = final_model.fit(disp=False)

# Resumen del modelo entrenado
print(final_result.summary())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import shapiro

# 1. Extraer los residuos del modelo
residuals = final_result.resid

# 2. Gráfica de residuos en el tiempo
plt.figure(figsize=(12,4))
plt.plot(residuals)
plt.title('Residuos del modelo')
plt.show()

# 3. Histograma de los residuos
plt.figure(figsize=(8,4))
sns.histplot(residuals, kde=True, bins=30)
plt.title('Histograma de los residuos')
plt.show()

# 4. Gráfica Q-Q para ver normalidad
import statsmodels.api as sm
sm.qqplot(residuals, line='s')
plt.title('Q-Q plot de los residuos')
plt.show()

# 5. Autocorrelación de residuos
fig, ax = plt.subplots(1,2, figsize=(16,4))
plot_acf(residuals, ax=ax[0])
plot_pacf(residuals, ax=ax[1])
plt.show()

# 6. Ljung-Box test
lb_test = acorr_ljungbox(residuals, lags=[10], return_df=True)
print("Ljung-Box test (lag=10):")
print(lb_test)

# 7. Shapiro-Wilk test para normalidad
shapiro_test = shapiro(residuals)
print(f"\nShapiro-Wilk Test:")
print(f"  - Estadístico: {shapiro_test[0]:.4f}")
print(f"  - p-value: {shapiro_test[1]:.4f}")

In [ ]:
# Cargar el CSV de los 2 días reales
df_2dias = pd.read_csv('../data/serie_temporal_semana1_predecir.csv')

# Asegurarte de que la columna timestamp sea datetime y esté ordenado
df_2dias['user_ts__'] = pd.to_datetime(df_2dias['user_ts__'])
df_2dias = df_2dias.sort_values('user_ts__')

# Establecer como índice el timestamp
df_2dias.set_index('user_ts__', inplace=True)

# Selecciona la columna de la variable objetivo
serie_real_pred = df_2dias['powerPerPreform_CurrentPreformNeckFinishTemperature.0']

# Eliminar nulos (por si las dudas)
serie_real_pred = serie_real_pred.dropna()

# Revisar el rango y el número de datos reales
print(serie_real_pred.index.min(), serie_real_pred.index.max(), len(serie_real_pred))

In [ ]:
# Cuántos pasos quieres pronosticar con SARIMA por minuto
steps_forecast = 2880  # 2 días en minutos

# Predicción con el modelo SARIMA entrenado en serie_minuto
pred_uc = final_result.get_forecast(steps=steps_forecast)

# Obtener la media del pronóstico y el intervalo de confianza
forecast_mean_minuto = pred_uc.predicted_mean
forecast_ci = pred_uc.conf_int()

# Creamos el índice correcto para el forecast por minuto
forecast_index_minuto = pd.date_range(
    start='2025-01-11 18:00:01+00:00',  # Aquí pones el inicio de la predicción (coherente con tu ventana)
    periods=steps_forecast,
    freq='T'  # Frecuencia de minutos
)

# Asignamos el índice al forecast y al intervalo de confianza
forecast_mean_minuto.index = forecast_index_minuto
forecast_ci.index = forecast_index_minuto

In [ ]:
# Reindexamos sin método para generar NaNs y después interpolamos
forecast_interpolado = forecast_mean_minuto.reindex(serie_real_pred.index)
forecast_interpolado = forecast_interpolado.interpolate(method='time')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,6))

# Serie real de los 2 días
plt.plot(serie_real_pred.index, serie_real_pred.values, label='Serie Real (segundos)', alpha=0.7)

# Forecast interpolado
plt.plot(forecast_interpolado.index, forecast_interpolado.values, label='Forecast SARIMA interpolado', color='red')

plt.title('Predicción SARIMA (Interpolada) vs Datos Reales; powerPerPreform_CurrentPreformNeckFinishTemperature.0')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Asegúrate de que ambos tienen el mismo tamaño
assert len(serie_real_pred) == len(forecast_interpolado)

# Calculamos los errores
mae = mean_absolute_error(serie_real_pred.values, forecast_interpolado.values)
rmse = np.sqrt(mean_squared_error(serie_real_pred.values, forecast_interpolado.values))

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

***SEMANA 2***

In [ ]:
#importamos el dia con el que trabajamos
df_sem2 = pd.read_csv('../data/serie_temporal_semana2.csv')

In [ ]:
# Convertimos el timestamp en índice de tiempo
df_sem2['user_ts__'] = pd.to_datetime(df_sem2['user_ts__'])
df_sem2.set_index('user_ts__', inplace=True)

In [ ]:
# Seleccionamos la variable con más datos
serie2 = df_sem2['powerPerPreform_CurrentPreformNeckFinishTemperature.0'].dropna()

# Graficamos la serie temporal
plt.figure(figsize=(14, 6))
plt.plot(serie2, color='blue')

plt.title('Serie temporal - powerPerPreform_CurrentPreformNeckFinishTemperature.0', fontsize=16)
plt.xlabel('Fecha y hora', fontsize=14)
plt.ylabel('Temperatura', fontsize=14)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Asegúrate de tener el índice en datetime y ordenado
serie2.index = pd.to_datetime(serie2.index)
serie2 = serie2.sort_index()

# Resampleamos cada 20 segundos y calculamos el promedio en cada ventana de 20 segundos
serie_30s = serie2.resample('30S').mean().dropna()

# Revisamos
print(f"Cantidad de datos resampleados: {len(serie_30s)}")
serie_30s.head()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Entrena el modelo con los mejores parámetros
final_model = SARIMAX(serie_30s,
                      order=(4, 1, 2),
                      seasonal_order=(0, 1, 2, 60),
                      enforce_stationarity=False,
                      enforce_invertibility=False)

final_result = final_model.fit(disp=False)

# Resumen del modelo entrenado
print(final_result.summary())

In [ ]:
# Cargar el CSV de los 2 días reales
df_2dias = pd.read_csv('../data/serie_temporal_semana2_predecir.csv')

# Asegurarte de que la columna timestamp sea datetime y esté ordenado
df_2dias['user_ts__'] = pd.to_datetime(df_2dias['user_ts__'])
df_2dias = df_2dias.sort_values('user_ts__')

# Establecer como índice el timestamp
df_2dias.set_index('user_ts__', inplace=True)

# Selecciona la columna de la variable objetivo
serie_real_pred = df_2dias['powerPerPreform_CurrentPreformNeckFinishTemperature.0']

# Eliminar nulos (por si las dudas)
serie_real_pred = serie_real_pred.dropna()

# Revisar el rango y el número de datos reales
print(serie_real_pred.index.min(), serie_real_pred.index.max(), len(serie_real_pred))

In [ ]:
# Cuántos pasos quieres pronosticar con SARIMA por minuto
steps_forecast = 2880  # 2 días en minutos

# Predicción con el modelo SARIMA entrenado en serie_minuto
pred_uc = final_result.get_forecast(steps=steps_forecast)

# Obtener la media del pronóstico y el intervalo de confianza
forecast_mean_minuto = pred_uc.predicted_mean
forecast_ci = pred_uc.conf_int()

# Creamos el índice correcto para el forecast por minuto
forecast_index_minuto = pd.date_range(
    start='2025-01-13 18:00:01+00:00',  # Aquí pones el inicio de la predicción (coherente con tu ventana)
    periods=steps_forecast,
    freq='T'  # Frecuencia de minutos
)

# Asignamos el índice al forecast y al intervalo de confianza
forecast_mean_minuto.index = forecast_index_minuto
forecast_ci.index = forecast_index_minuto

In [ ]:
# Reindexamos sin método para generar NaNs y después interpolamos
forecast_interpolado = forecast_mean_minuto.reindex(serie_real_pred.index)
forecast_interpolado = forecast_interpolado.interpolate(method='time')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,6))

# Serie real de los 2 días
plt.plot(serie_real_pred.index, serie_real_pred.values, label='Serie Real (segundos)', alpha=0.7)

# Forecast interpolado
plt.plot(forecast_interpolado.index, forecast_interpolado.values, label='Forecast SARIMA interpolado', color='red')

plt.title('Predicción SARIMA (Interpolada) vs Datos Reales; powerPerPreform_CurrentPreformNeckFinishTemperature.0')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

# Crear un DataFrame para alinear las dos series y limpiar NaN
df_comparacion = pd.DataFrame({
    'real': serie_real_pred,
    'predicho': forecast_interpolado
})

# Eliminar filas con NaN en cualquiera de las dos series
df_comparacion = df_comparacion.dropna()

# Confirmamos tamaños nuevamente (por si acaso)
assert len(df_comparacion['real']) == len(df_comparacion['predicho'])

# Calculamos las métricas
mae = mean_absolute_error(df_comparacion['real'], df_comparacion['predicho'])
rmse = np.sqrt(mean_squared_error(df_comparacion['real'], df_comparacion['predicho']))

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

***SEMANA 3***

In [ ]:
#importamos el dia con el que trabajamos
df_sem3 = pd.read_csv('../data/serie_temporal_semana3.csv')

In [ ]:
# Convertimos el timestamp en índice de tiempo
df_sem3['user_ts__'] = pd.to_datetime(df_sem3['user_ts__'])
df_sem3.set_index('user_ts__', inplace=True)

In [ ]:
# Seleccionamos la variable con más datos
serie3 = df_sem3['powerPerPreform_CurrentPreformNeckFinishTemperature.0'].dropna()

# Graficamos la serie temporal
plt.figure(figsize=(14, 6))
plt.plot(serie3, color='blue')

plt.title('Serie temporal - powerPerPreform_CurrentPreformNeckFinishTemperature.0', fontsize=16)
plt.xlabel('Fecha y hora', fontsize=14)
plt.ylabel('Temperatura', fontsize=14)

plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Asegúrate de tener el índice en datetime y ordenado
serie3.index = pd.to_datetime(serie3.index)
serie3 = serie3.sort_index()

# Resampleamos cada 20 segundos y calculamos el promedio en cada ventana de 20 segundos
serie_30s = serie3.resample('30S').mean().dropna()

# Revisamos
print(f"Cantidad de datos resampleados: {len(serie_30s)}")
serie_30s.head()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Entrena el modelo con los mejores parámetros
final_model = SARIMAX(serie_30s,
                      order=(4, 1, 2),
                      seasonal_order=(0, 1, 2, 60),
                      enforce_stationarity=False,
                      enforce_invertibility=False)

final_result = final_model.fit(disp=False)

# Resumen del modelo entrenado
print(final_result.summary())

In [ ]:
# Cargar el CSV de los 2 días reales
df_2dias = pd.read_csv('../data/serie_temporal_semana3_predecir.csv')

# Asegurarte de que la columna timestamp sea datetime y esté ordenado
df_2dias['user_ts__'] = pd.to_datetime(df_2dias['user_ts__'])
df_2dias = df_2dias.sort_values('user_ts__')

# Establecer como índice el timestamp
df_2dias.set_index('user_ts__', inplace=True)

# Selecciona la columna de la variable objetivo
serie_real_pred = df_2dias['powerPerPreform_CurrentPreformNeckFinishTemperature.0']

# Eliminar nulos (por si las dudas)
serie_real_pred = serie_real_pred.dropna()

# Revisar el rango y el número de datos reales
print(serie_real_pred.index.min(), serie_real_pred.index.max(), len(serie_real_pred))

In [ ]:
# Cuántos pasos quieres pronosticar con SARIMA por minuto
steps_forecast = 2880  # 2 días en minutos

# Predicción con el modelo SARIMA entrenado en serie_minuto
pred_uc = final_result.get_forecast(steps=steps_forecast)

# Obtener la media del pronóstico y el intervalo de confianza
forecast_mean_minuto = pred_uc.predicted_mean
forecast_ci = pred_uc.conf_int()

# Creamos el índice correcto para el forecast por minuto
forecast_index_minuto = pd.date_range(
    start='2025-01-15 18:00:01+00:00',  # Aquí pones el inicio de la predicción (coherente con tu ventana)
    periods=steps_forecast,
    freq='T'  # Frecuencia de minutos
)

# Asignamos el índice al forecast y al intervalo de confianza
forecast_mean_minuto.index = forecast_index_minuto
forecast_ci.index = forecast_index_minuto

In [ ]:
# Reindexamos sin método para generar NaNs y después interpolamos
forecast_interpolado = forecast_mean_minuto.reindex(serie_real_pred.index)
forecast_interpolado = forecast_interpolado.interpolate(method='time')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15,6))

# Serie real de los 2 días
plt.plot(serie_real_pred.index, serie_real_pred.values, label='Serie Real (segundos)', alpha=0.7)

# Forecast interpolado
plt.plot(forecast_interpolado.index, forecast_interpolado.values, label='Forecast SARIMA interpolado', color='red')

plt.title('Predicción SARIMA (Interpolada) vs Datos Reales; powerPerPreform_CurrentPreformNeckFinishTemperature.0')
plt.xlabel('Fecha')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd

# Crear un DataFrame para alinear las dos series y limpiar NaN
df_comparacion = pd.DataFrame({
    'real': serie_real_pred,
    'predicho': forecast_interpolado
})

# Eliminar filas con NaN en cualquiera de las dos series
df_comparacion = df_comparacion.dropna()

# Confirmamos tamaños nuevamente (por si acaso)
assert len(df_comparacion['real']) == len(df_comparacion['predicho'])

# Calculamos las métricas
mae = mean_absolute_error(df_comparacion['real'], df_comparacion['predicho'])
rmse = np.sqrt(mean_squared_error(df_comparacion['real'], df_comparacion['predicho']))

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")

***SEMANA 4***